In [19]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, text
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
load_dotenv()
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)

In [21]:
# select 26 rows from SQL table to insert in dataframe.
query = text('''
            select c.ContactPersoonId, i.CampagneId, i.CampagneNaam, ca.Startdatum, a.Ondernemingsaard, a.Ondernemingstype, a.PrimaireActiviteit, f.Naam as Functie
            from Contactfiche c
            join Account a on a.AccountId = c.AccountId
            join Inschrijving i on i.ContactficheId = c.ContactPersoonId
            join Campagne ca on ca.CampagneId = i.CampagneId
            join ContactficheFunctie cf on cf.ContactpersoonId = c.ContactPersoonId
            join Functie f on f.FunctieId = cf.FunctieId
            where i.CampagneId is not null and a.status = 'actief';
            ''')
result = conn.execute(query)

#convert to dataframe
df_omschrijving = pd.DataFrame(result.fetchall())
df_omschrijving.set_index('ContactPersoonId', inplace=True)
df_omschrijving["rating"] = 5

query2 = text('''
                with pageview_count as(
                    select p.PageTitle, count(p.ContactId) as count
                    from Pageviews p
                    group by p.PageTitle
                )
                select PageTitle, ContactId
                from Pageviews
                where Pagetitle in (select top (2500) PageTitle
                from pageview_count
                order by count desc)
              ''')

result2 = conn.execute(query2)
df_pageviews = pd.DataFrame(result2.fetchall())
df_pageviews.set_index('ContactId', inplace=True)
df_pageviews["rating"] = 1


df_omschrijving.head()

,CampagneId,CampagneNaam,Startdatum,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,,
9E22A289-C6F3-ED11-8849-6045BD895CDC,044F17B7-A8B6-ED11-83FF-6045BD895CDC,OV-JO-Stamgasten 2023,2023-05-23 19:00:00,Diensten,Bedrijf,Financiële diensten,Verantwoordelijke Financieel,5
12C6CFE5-5020-EC11-8124-001DD8B72B61,0A14941A-711C-EC11-8123-001DD8B72B61,OV-Community Bouw en Vastgoed 2022,2022-01-31 08:00:00,Diensten,Familiebedrijf,Bouw,Verantwoordelijke Commercieel,5
EC6D5B41-BA4A-E711-80E7-001DD8B72B61,A534E7FC-EEF3-E711-80EE-001DD8B72B62,OV-NW-Voka Update: The Big Refresh 4,2018-02-26 06:00:00,Diensten,Bedrijf,Grafische industrie en diensten,Bedrijfsleider,5
FC779EEB-7775-E911-80FE-001DD8B72B62,DF63BB61-A6AA-EC11-983F-0022488488D8,OV-Haven-Werkgroep milieu en energie-stikstof ...,2022-05-18 09:00:00,Diensten,Bedrijf,Overige industrie & diensten,"Verantwoordelijke Duurzaamheid, Milieu & Energie",5
01218324-0942-E611-80D6-005056B06EC4,AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018,2018-01-18 06:00:00,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,5


In [22]:
#remove the rows  [CampagneId, CampagneNaam, Startdatum] from df_omschrijving into a new dataframe
df_campagne = df_omschrijving.iloc[:, 0:3]
df_campagne = df_campagne[df_campagne['Startdatum'] > '2022-11-06']
df_campagne['rating'] = 10
df_omschrijving.drop(df_omschrijving.columns[[0, 1, 2]], axis=1, inplace=True)
df_omschrijving.drop_duplicates(inplace=True)
df_campagne

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
9E22A289-C6F3-ED11-8849-6045BD895CDC,044F17B7-A8B6-ED11-83FF-6045BD895CDC,OV-JO-Stamgasten 2023,2023-05-23 19:00:00,10
338A722F-9748-ED11-BBA1-6045BD895CDC,7E04D7C8-59FC-EC11-82E5-000D3A3A954E,OV-NW-Voka Update-Big Refresh-voorjaar 2023,2023-02-06 08:30:00,10
AB8E5FAB-126B-ED11-9561-6045BD895BFB,A02595A3-8A5F-ED11-9561-6045BD8952CE,OV-NW- Quick Refresh - webinar: Arbeidsdeal me...,2022-11-29 12:00:00,10
01536C51-37E4-ED11-A7C7-6045BD895FE3,29D3979C-A3B9-ED11-83FF-6045BD8956C9,OV-Financieel management voor niet-financiëlen...,2023-06-01 09:00:00,10
85A2204B-733D-EC11-8125-001DD8B72B61,A8949A21-6CEE-ED11-8849-6045BD8956C9,OV-NW&IN-EDIH-DIGITALIS-XpertFinder-Data-AI,2023-10-19 08:00:00,10
...,...,...,...,...
765D8544-4BF1-E811-80FA-001DD8B72B62,317CD023-2B1E-ED11-B83D-000D3AAD783A,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,2022-12-21 19:00:00,10
06DB6653-AA1E-ED11-B83D-000D3AAD783A,40F621C9-2455-ED11-BBA1-6045BD895BFB,OV-NW-Voka Cluster Sifferdok,2022-12-06 09:30:00,10
06DB6653-AA1E-ED11-B83D-000D3AAD783A,01FF068A-26D5-ED11-A7C7-6045BD895FE3,OV-NW-Voka|VeGHO Portlunch met Grégoire Dallem...,2023-09-22 14:00:00,10


In [23]:
df_omschrijving

,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,
9E22A289-C6F3-ED11-8849-6045BD895CDC,Diensten,Bedrijf,Financiële diensten,Verantwoordelijke Financieel,5
12C6CFE5-5020-EC11-8124-001DD8B72B61,Diensten,Familiebedrijf,Bouw,Verantwoordelijke Commercieel,5
EC6D5B41-BA4A-E711-80E7-001DD8B72B61,Diensten,Bedrijf,Grafische industrie en diensten,Bedrijfsleider,5
FC779EEB-7775-E911-80FE-001DD8B72B62,Diensten,Bedrijf,Overige industrie & diensten,"Verantwoordelijke Duurzaamheid, Milieu & Energie",5
01218324-0942-E611-80D6-005056B06EC4,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,5
...,...,...,...,...,...
0896513A-FBFE-E411-ABE8-005056B06EB4,Diensten,Bedrijf,Financiële diensten,Verantwoordelijke Kwaliteit,5
C8BD4C47-1BE3-EC11-BB3C-000D3ABCAA04,Productie,Familiebedrijf,Bouw,Medewerker Innovatie / Onderzoek & Ontwikkeling,5
C0739812-8611-ED11-B83D-000D3A43E419,None,Bedrijf,Consultancy,Medewerker Kwaliteit,5


In [24]:
df_pageviews.head()

,PageTitle,rating
ContactId,,
71C7CDFA-379C-EB11-811E-001DD8B72B62,Weer hogere loonindexering op komst in 2024 | ...,1
2A04879B-1141-EC11-8125-001DD8B72B61,Nieuwjaarsinvitito 2023 | Voka,1
1E9BCBC0-9025-EC11-8124-001DD8B72B61,Netwerkevents | Voka,1
3F1EB182-8E3C-E811-80EF-001DD8B72B62,Marketing | Voka,1
5073196D-6611-E811-80EF-001DD8B72B61,Bedrijfsafval: dit verandert op 1 januari | Voka,1


In [25]:
df_pivot_omschrijving = pd.pivot_table(df_omschrijving, index='ContactPersoonId', columns=['Ondernemingsaard', 'Ondernemingstype', 'PrimaireActiviteit', 'Functie'], values='rating', fill_value = 0)
df_pivot_omschrijving

Ondernemingsaard                                      Diensten             \
Ondernemingstype                                       Bedrijf              
PrimaireActiviteit                   Accountancy & boekhouding              
Functie                                         Bedrijfsleider Bestuurder   
ContactPersoonId                                                            
00223C8E-467F-E311-BBFD-005056B06EB4                         0          0   
0036CCCE-B36F-E111-B43A-00505680000A                         0          0   
003740F4-F4A0-EB11-811E-001DD8B72B62                         0          0   
005604AC-395A-EB11-8117-001DD8B72B61                         0          0   
00AD7781-9403-EE11-8F6E-6045BD895554                         0          0   
...                                                        ...        ...   
FF98CB50-05B3-EB11-811F-001DD8B72B62                         0          0   
FF9ACC3A-5FA7-EB11-811E-001DD8B72B62                         0          0   
FFAB5F2A-32F7-EA11-8115-001DD8B72B62                         0          0   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                         0          0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                         0          0   

Ondernemingsaard                                           \
Ondernemingstype                                            
PrimaireActiviteit                                          
Functie                              Contact Lidmaatschap   
ContactPersoonId                                            
00223C8E-467F-E311-BBFD-005056B06EB4                    0   
0036CCCE-B36F-E111-B43A-00505680000A                    0   
003740F4-F4A0-EB11-811E-001DD8B72B62                    0   
005604AC-395A-EB11-8117-001DD8B72B61                    0   
00AD7781-9403-EE11-8F6E-6045BD895554                    0   
...                                                   ...   
FF98CB50-05B3-EB11-811F-001DD8B72B62                    0   
FF9ACC3A-5FA7-EB11-811E-001DD8B72B62                    0   
FFAB5F2A-32F7-EA11-8115-001DD8B72B62                    0   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                    0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                    0   

Ondernemingsaard                                           \
Ondernemingstype                                            
PrimaireActiviteit                                          
Functie                              Directie of kaderlid   
ContactPersoonId                                            
00223C8E-467F-E311-BBFD-005056B06EB4                    0   
0036CCCE-B36F-E111-B43A-00505680000A                    0   
003740F4-F4A0-EB11-811E-001DD8B72B62                    0   
005604AC-395A-EB11-8117-001DD8B72B61                    0   
00AD7781-9403-EE11-8F6E-6045BD895554                    0   
...                                                   ...   
FF98CB50-05B3-EB11-811F-001DD8B72B62                    0   
FF9ACC3A-5FA7-EB11-811E-001DD8B72B62                    0   
FFAB5F2A-32F7-EA11-8115-001DD8B72B62                    0   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                    0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                    0   

Ondernemingsaard                                                      \
Ondernemingstype                                                       
PrimaireActiviteit                                                     
Functie                              Management Assistent Medewerker   
ContactPersoonId                                                       
00223C8E-467F-E311-BBFD-005056B06EB4                    0          0   
0036CCCE-B36F-E111-B43A-00505680000A                    0          0   
003740F4-F4A0-EB11-811E-001DD8B72B62                    0          0   
005604AC-395A-EB11-8117-001DD8B72B61                    0          0   
00AD7781-9403-EE11-8F6E-6045BD895554                    0          0   
...                                                   ...        ...   
FF98CB50-05B3-EB11-81

In [26]:
df_pivot_campagne = pd.pivot_table(df_campagne, index='ContactPersoonId', columns=['CampagneId'], values='rating', fill_value = 0)

In [27]:
df_pivot_pageviews = pd.pivot_table(df_pageviews, index='ContactId', columns=['PageTitle'], values='rating', fill_value = 0)
df_pivot_pageviews

PageTitle,"""Aansluiten bij een lerend netwerk van Voka is de investering meer dan waard!"" | Voka","""Al 100 hectare industriegebied aan Albertkanaal geschrapt"" | Voka","""Bij drie op tien bedrijven zijn loonkosten tot 10% gestegen"" | Voka","""Bijkomende droogtemaatregelen zijn nefast voor Limburgse ondernemingen"" | Voka","""Bouwen doe je met een blik op 10 jaar verder"" | Voka","""De Luchtige Kant"": de kalkoen van Suzy Cretskens | Voka","""Een authentieke cultuur start voor ons bij leiderschap"" | Voka","""FIRST Tech Challenge als middel tegen mismatch op de arbeidsmarkt? | Voka","""Het VCDO-traject van Voka geeft richting en structuur aan ons duurzaamheidsbeleid."" | Voka","""Keep calm and go international."" | Voka",...,hier voor de nieuwe folder van 2022.,inhoud 'Gun jezelf en je onderneming een BTW-update | Voka,jaarverslag 2022 | Voka,vokaadmin | Voka,vzw - interesse Bryo ScaleUp | Voka,vzw - interesse Bryo StandUp | Voka,vzw - interesse Bryo StartUp | Voka,vzw - interesse Mercurio Go International | Voka,vzw - interesse Tech@venture | Voka,| Voka
ContactId,,,,,,,,,,,,,,,,,,,,,
00013C8A-6F1C-E211-9DAA-005056B06EB4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00017E48-212C-EC11-8127-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002EDD7-2405-E511-ABE8-005056B06EB4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006CEC0-931A-EB11-8117-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000C5DD4-016A-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FF76B927-E283-E611-80E3-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FF9A9C90-DC1A-E711-80E4-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FFD3EBB0-F16C-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#join both dataframes
#df_pivot = df_pivot_omschrijving.join(df_pivot_pageviews, how='inner')
df_pivot = pd.concat([df_pivot_omschrijving, df_pivot_pageviews, df_pivot_campagne], axis=1, join='inner')
df_pivot.head()

,"(Diensten, Bedrijf, Accountancy & boekhouding, Bedrijfsleider)","(Diensten, Bedrijf, Accountancy & boekhouding, Bestuurder)","(Diensten, Bedrijf, Accountancy & boekhouding, Contact Lidmaatschap)","(Diensten, Bedrijf, Accountancy & boekhouding, Directie of kaderlid)","(Diensten, Bedrijf, Accountancy & boekhouding, Management Assistent)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Commercieel)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Communicatie)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Financieel)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Juridische dienst)",...,FC0BCA85-5EE3-ED11-A7C7-6045BD8F0F71,FCA5B3B1-525F-ED11-9561-6045BD8956C9,FD106E55-5AEA-ED11-8849-6045BD895499,FD27C35D-0990-ED11-AAD1-6045BD895BFB,FEC6B604-CEC4-ED11-83FF-6045BD9745FB,FEE2E778-15A2-ED11-AAD0-6045BD895BFB,FEE8099B-2DA2-ED11-AAD1-6045BD895D85,FF35136E-0CF0-ED11-8849-6045BD9745FB,FFCA2662-1125-ED11-B83D-000D3A2D2C74,FFE56B2D-B848-EE11-BE6E-6045BD895D78
01B45481-0877-E911-80FE-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01FA3FD7-FC20-EC11-8124-001DD8B72B61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0234BCDF-8D6C-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
028EB781-26D8-EC11-A7B5-000D3ABD121D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
049D9DA6-2A47-E511-8CF9-005056B06EC4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
similarity_matrix = cosine_similarity(df_pivot, [df_pivot.loc["01B45481-0877-E911-80FE-001DD8B72B62"]]).reshape(1,-1)[0]
similarity_matrix

array([1.        , 0.        , 0.21796478, 0.        , 0.00335056,
       0.        , 0.3253531 , 0.00217637, 0.        , 0.01184075,
       0.16849679, 0.00316961, 0.00338077, 0.        , 0.        ,
       0.22533003, 0.00259073, 0.18869201, 0.00184174, 0.        ,
       0.00275374, 0.00351035, 0.18690669, 0.        , 0.        ,
       0.        , 0.00677999, 0.        , 0.35877396, 0.00343086,
       0.        , 0.5233266 , 0.        , 0.        , 0.41956296,
       0.        , 0.        , 0.        , 0.        , 0.26418074,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.0070207 , 0.00681108, 0.        , 0.35103497,
       0.0014193 , 0.00550162, 0.        , 0.        , 0.        ,
       0.35103497, 0.14646913, 0.        , 0.        , 0.00343086,
       0.00209901, 0.18951427, 0.1475552 , 0.        , 0.12616132,
       0.        , 0.00259626, 0.00681108, 0.25946441, 0.        ,
       0.00216022, 0.        , 0.25962556, 0.        , 0.     

In [30]:
select_contact = "C39094D7-60B1-EB11-811F-001DD8B72B62"

similarities = df_pivot.index.join(similarity_matrix)
similarities = pd.DataFrame({'ContactPersoonId':df_pivot.index, 'sim':similarity_matrix}).set_index('ContactPersoonId')

similar_users = similarities[similarities['sim'] > 0.5].sort_values(by='sim', ascending=False)
similar_users

,sim
ContactPersoonId,
01B45481-0877-E911-80FE-001DD8B72B62,1.000000
19C883F7-877C-ED11-81AD-6045BD895D85,0.523327


In [31]:
done_campaigns = df_campagne.loc[df_campagne.index == select_contact]['CampagneId']
done_campaigns

ContactPersoonId
C39094D7-60B1-EB11-811F-001DD8B72B62    D143743E-7B92-ED11-AAD1-6045BD895B5A
C39094D7-60B1-EB11-811F-001DD8B72B62    D143743E-7B92-ED11-AAD1-6045BD895B5A
C39094D7-60B1-EB11-811F-001DD8B72B62    D143743E-7B92-ED11-AAD1-6045BD895B5A
Name: CampagneId, dtype: object

In [32]:
similar_campagnes = df_campagne[df_campagne.index.isin(similar_users.index)].replace(0, np.nan).dropna(axis=1, how='all')
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,A2F63005-8EA8-ED11-AAD1-6045BD895D85,OV-NW-Voka Bilan 2023,2023-05-25 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,317CD023-2B1E-ED11-B83D-000D3AAD783A,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,2022-12-21 19:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,6D357184-F5AF-EC11-9840-000D3A278F96,OV-P-GROEP H-SCALE UP-BRYO 2023,2023-05-30 09:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,E1B0A3B0-8A4F-ED11-BBA2-6045BD895BFB,OV-NW-Voka|VeGHO Portlunch met Koen Schoors,2022-12-15 11:30:00,10
01B45481-0877-E911-80FE-001DD8B72B62,6D357184-F5AF-EC11-9840-000D3A278F96,OV-P-GROEP H-SCALE UP-BRYO 2023,2023-05-30 09:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,E1B0A3B0-8A4F-ED11-BBA2-6045BD895BFB,OV-NW-Voka|VeGHO Portlunch met Koen Schoors,2022-12-15 11:30:00,10
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,C693DACA-04CF-ED11-B597-6045BD895805,OV-NW-Rode Loper Selectie-Gent Jazz-Gregory Po...,2023-07-05 19:00:00,10


In [33]:
#remove double campagneIds in similar_campagnes
similar_campagnes = similar_campagnes.drop_duplicates(subset=['CampagneId'])
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,A2F63005-8EA8-ED11-AAD1-6045BD895D85,OV-NW-Voka Bilan 2023,2023-05-25 16:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,317CD023-2B1E-ED11-B83D-000D3AAD783A,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,2022-12-21 19:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,6D357184-F5AF-EC11-9840-000D3A278F96,OV-P-GROEP H-SCALE UP-BRYO 2023,2023-05-30 09:00:00,10
01B45481-0877-E911-80FE-001DD8B72B62,E1B0A3B0-8A4F-ED11-BBA2-6045BD895BFB,OV-NW-Voka|VeGHO Portlunch met Koen Schoors,2022-12-15 11:30:00,10
01B45481-0877-E911-80FE-001DD8B72B62,C693DACA-04CF-ED11-B597-6045BD895805,OV-NW-Rode Loper Selectie-Gent Jazz-Gregory Po...,2023-07-05 19:00:00,10


In [34]:
#only show campagnes that will start in the future
similar_campagnes = similar_campagnes[similar_campagnes['Startdatum'] > '2023-11-06']
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,rating
ContactPersoonId,,,,
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,10
